In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from skimage.color import rgb2gray
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf


In [2]:
dataset=[]
for dirname, _, filenames in os.walk('C:/Users/Muhammad Al-Barbary/OneDrive/Desktop/Education/One-Lab Intern/Melanoma Detection/PH2Dataset/PH2 Dataset images'):
    for filename in filenames:
        image = Image.open(os.path.join(dirname, filename))
        image=image.resize((575, 765))
        image= np.asarray(image)
        if (len(image.shape) == 3):
            dataset.append(rgb2gray(image))
        


In [3]:
df = pd.read_csv('C:/Users/Muhammad Al-Barbary/OneDrive/Desktop/Education/One-Lab Intern/Melanoma Detection/PH2Dataset/PH2_dataset.csv')[12:]

In [4]:
labels=[]
for i in range(0,200):
    if (np.asarray(df)[i][4]=='X'):
        labels.append(1)
    else:
        labels.append(0)
labels=np.asarray(labels)

In [5]:
dataset_flat=[]
for i in range(len(dataset)):
    dataset_flat.append(np.asarray(dataset[i]).flatten())
dataset_flat=np.asarray(dataset_flat)

In [6]:
def SVMTrain(dataset_flat,labels,components):
    PCA_Model=PCA(n_components=components)
    X_Train_PCA=PCA_Model.fit_transform(dataset_flat)
    X_train, X_test, y_train, y_test = train_test_split(X_Train_PCA, labels, test_size = 0.25, random_state = 0)
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    test_accuracy=accuracy_score(y_test,y_pred)
    y_pred_train = classifier.predict(X_train)
    train_accuracy=accuracy_score(y_train,y_pred_train)
    return components,train_accuracy,test_accuracy

In [7]:
PCA_ComponentsNumber,TrainAccuracy,TestAccuracy=SVMTrain(dataset_flat,labels,10)
print(f"Training Accuracy= {TrainAccuracy},Testing Accuracy= {TestAccuracy} at {PCA_ComponentsNumber} PCA Components")

Training Accuracy= 0.8866666666666667,Testing Accuracy= 0.88 at 10 PCA Components


In [41]:
model= Sequential(
    [
        Dense(15,activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1)),
        Dense(10,activation='relu',  kernel_regularizer=tf.keras.regularizers.L2(0.1)),
        Dense(1,activation='sigmoid',  kernel_regularizer=tf.keras.regularizers.L2(0.1))
    ],name='my_model'
)

In [42]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

In [43]:
PCA_Model=PCA(n_components=10)
X_Train_PCA=PCA_Model.fit_transform(dataset_flat)
X_train, X_test, y_train, y_test = train_test_split(X_Train_PCA, labels, test_size = 0.25, random_state = 0)
model.fit(X_train,y_train,epochs=500)


Epoch 1/500
5/5 [==============================] - 0s 2ms/step - loss: 4.7590
Epoch 2/500
5/5 [==============================] - 0s 1ms/step - loss: 4.4817
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 4.2406
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 4.0534
Epoch 5/500
5/5 [==============================] - 0s 1ms/step - loss: 3.8338
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 3.6469
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 3.4695
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 3.3036
Epoch 9/500
5/5 [==============================] - 0s 1ms/step - loss: 3.1525
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0150
Epoch 11/500
5/5 [==============================] - 0s 3ms/step - loss: 2.8788
Epoch 12/500
5/5 [==============================] - 0s 2ms/step - loss: 2.7450
Epoch 13/500
5/5 [==============================] - 0s 2ms/st

In [44]:
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    for i in range (len(y_pred)):
        if y_pred[i]>0.5:
            y_pred[i]=1
        else:
            y_pred[i]=0
    for i in range (len(y_pred_train)):
        if y_pred_train[i]>0.5:
            y_pred_train[i]=1
        else:
            y_pred_train[i]=0

5/5 [==============================] - 0s 2ms/step


In [47]:
sum_train=0
for i in range (len(y_pred_train)):
    if (y_pred_train[i]==y_train[i]):
        sum_train+=1
print(f"Accuracy of training set is {sum_train/len(y_pred_train)}")

Accuracy of training set is 0.92


In [48]:
sum_train=0
for i in range (len(y_pred)):
    if (y_pred[i]==y_test[i]):
        sum_train+=1
print(f"Accuracy of testing set is {sum_train/len(y_pred)}")

Accuracy of testing set is 0.88
